# Penguin Bit-By-Bit

Let's build a game bit by bit!

Bear, um, with us.
The following lines are the only boilerplate,
just execute the cell
(by clicking CTRL-Enter)
and move on.

In [1]:
import asyncio
from twisted.internet import asyncioreactor
asyncioreactor.install(asyncio.get_event_loop())
from twisted.internet import reactor
import ppb
import txppb
__file__ = None

pygame 1.9.6
Hello from the pygame community. https://www.pygame.org/contribute.html


Next, we need a
"setup"
function.
A setup function traditionally sets up the game elements.
However,
our function will just put the game
"scene"
in a global variable.

In [2]:
def setup(scene):
    global SCENE
    SCENE = scene

Wondeful! Now let's start the game loop.
This will open a window with nothing in it.

In [3]:
d = txppb.run(setup, reactor)
d.addBoth(print)

The game is called "penguin bit by bit",
so we should probably have a penguin.
In PPB,
these are called "sprites".
This is the standard terminology for video games.

In [4]:
class Penguin(ppb.Sprite):
    pass

We defined a penguin.
In PPB, game objects are called "sprites".
This is common in video game development.

In PPB, sprites automatically find the image if it is in the current directory,
and named the same as the sprite.
I made sure I had a cute little png named `penguin.png`.

Now let's put the penguin riiiiiight in the middle.

In [5]:
SCENE.add(Penguin(pos=(0,0)))

But penguins are not meant to sit still! The penguin should move around.
We have the player control the penguin with the arrow keys.
First, let's map the keys to vectors:

In [6]:
from ppb import keycodes

DIRECTIONS = {keycodes.Left: ppb.Vector(-1,0), keycodes.Right: ppb.Vector(1,0),
              keycodes.Up: ppb.Vector(0, 1), keycodes.Down: ppb.Vector(0, -1)}

Next let's make sure the penguin has a sense of direction, and it follows it:

In [7]:
Penguin.direction = ppb.Vector(0, 0)

def Penguin_on_update(self, update_event, signal):
    self.position += update_event.time_delta * self.direction
 
Penguin.on_update = Penguin_on_update

Oh, um, well.
The penguin is diligently moving...at zero speed,
precisely nowhere.
Let's manually set the direction to see what happens.

In [8]:
Penguin.direction = DIRECTIONS[keycodes.Up]/4

The direction is up, but a little slowly. This gives enough time to set the penguin's direction back to zero manually.
Let's do that now!

In [9]:
Penguin.direction = ppb.Vector(0, 0)

Phew, that was exciting -- but not what we wanted.
We want the penguin to respond to key presses.
Let's set it to set the direction to the key press,
and back to zero when the key is released.

In [10]:
def Penguin_on_key_pressed(self, key_event, signal):
    self.direction = DIRECTIONS.get(key_event.key, ppb.Vector(0, 0))    

def Penguin_on_key_released(self, key_event, signal):
    if key_event.key in DIRECTIONS:
        self.direction = ppb.Vector(0, 0)

Penguin.on_key_pressed = Penguin_on_key_pressed
Penguin.on_key_released = Penguin_on_key_released

The Penguin is a bit bored, isn't it?
Maybe we should give it an orange ball to play with.

In [11]:
class OrangeBall(ppb.Sprite):
    pass

Again, I made sure to have an image called `orangeball.png`.
Now let's put the ball on the left side of the screen.

In [12]:
SCENE.add(OrangeBall(pos=(-4, 0)))

Try as it might, the penguin cannot kick the ball.
Let's have the ball move away from the penguin when it approaches.
First, we define a "smooth step" function.
This is a trick of the art in video game development,
this function gives the illusion of smoothness.

In [13]:
def smooth_step(t):
    return t * t * (3 - 2 * t)

OK, now we need to add movement code to the ball.
Unless the ball is already in motion,
if it sees the penguin approaching,
it will move in the opposite direction.
(If the penguin is on top of it, it will move in a random direction.)

It checks where it needs to go,
the "target position",
and then gets to the target position over a minute.
It will use the smooth step function.
When it is done, it will mark itself as "not moving".

In [14]:
import random
OrangeBall.is_moving = False
def OrangeBall_on_update(self, update_event,signal):
    if not self.is_moving:
        penguin, = update_event.scene.get(kind=Penguin)
        if (max(penguin.bottom, self.bottom) <= min(penguin.top, self.top) and
            max(penguin.left, self.left) <= min(penguin.right, self.right)):
            try:
                direction = (self.position - penguin.position).normalize()
            except ZeroDivisionError:
                direction = ppb.Vector(random.uniform(-1, 1), random.uniform(-1, 1)).normalize()
            self.target_position = self.position + direction
            self.original_position = self.position
            self.time_passed = 0
            self.is_moving = True
    else:
        self.time_passed += update_event.time_delta
        if self.time_passed >= 1:
            self.position = self.target_position
            self.is_moving = False
            return
        t = smooth_step(self.time_passed)
        self.position = (1-t) * self.original_position + t * self.target_position
OrangeBall.on_update = OrangeBall_on_update

But just kicking a ball around is not that much fun. Let's add a target.

In [15]:
class Target(ppb.Sprite):
    pass

Let's put the target at the right of the screen.

In [16]:
SCENE.add(Target(pos=(4, 0)))

Now, we will want a reward for the penguin,
when it kicks the ball into the target.
How about a fish?

In [17]:
class Fish(ppb.Sprite):
    pass

When the target gets the ball, it should remove it, and create a new ball at the other end of the screen.
Then, it will cause a fish to appear.

In [18]:
def Target_on_update(self, update_event, signal):
    for ball in update_event.scene.get(kind=OrangeBall):
        if (max(ball.bottom, self.bottom) <= min(ball.top, self.top) and
            max(ball.left, self.left) <= min(ball.right, self.right)):
            update_event.scene.remove(ball)
            update_event.scene.add(OrangeBall(pos=(-4, random.uniform(-3, 3))))
            update_event.scene.add(Fish(pos=(random.uniform(-4, -3), random.uniform(-3, 3))))
Target.on_update = Target_on_update

We want to have the penguin eat the fish.
When the fish sees the penguin, it should vanish.

In [19]:
def Fish_on_update(self, update_event,signal):
    penguin, = update_event.scene.get(kind=Penguin)
    if (max(penguin.bottom, self.bottom) <= min(penguin.top, self.top) and
        max(penguin.left, self.left) <= min(penguin.right, self.right)):
        update_event.scene.remove(self)
Fish.on_update = Fish_on_update